In [2]:
import pandas as pd 
import numpy as np 

In [8]:
sales_train = pd.read_csv('sales_train.csv')
test = pd.read_csv('test.csv')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

In [10]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [11]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [12]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [13]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [45]:
shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [23]:
sales_train = pd.read_csv('sales_train.csv')

In [24]:
sales_train = pd.merge(sales_train,items,how='left',on='item_id')
sales_train = pd.merge(sales_train,item_categories,how='left',on='item_category_id')
sales_train = pd.merge(sales_train,shops,how='left',on ='shop_id')

In [26]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [31]:
sales_train.dtypes

date                   object
date_block_num          int64
shop_id                 int64
item_id                 int64
item_price            float64
item_cnt_day          float64
item_name              object
item_category_id        int64
item_category_name     object
shop_name              object
dtype: object

In [37]:
##break date into day month year 
sales_train['date'] = pd.to_datetime(sales_train['date'],format='%d.%m.%Y')


In [43]:
sales_train['day'] = sales_train['date'].map(lambda x: x.day)
sales_train['month'] = sales_train['date'].map(lambda x: x.month)
sales_train['year'] = sales_train['date'].map(lambda x: x.year)

In [79]:
#potential features 
#price change in percent
#price change relative to cateogory 
#price relative to other stores 
sales_train['sales'] = sales_train['item_price']*sales_train['item_cnt_day']
sales_monthly_train = sales_train.groupby(['year','month','shop_id','item_id'],as_index=False)['sales'].sum()

In [80]:
price_end_month = sales_train[['year','month','shop_id','item_id','item_price']].drop_duplicates(subset=['year','month','shop_id','item_id'],keep='last')

In [81]:
price_end_month.sort_values(['year','month'],inplace=True)
price_end_month['last_price'] = price_end_month.groupby(['shop_id','item_id'])['item_price'].shift(1)


In [82]:
sales_monthly_train = pd.merge(sales_monthly_train,price_end_month,how='left',on=['year','month','shop_id','item_id'])

In [89]:
avg_price = sales_monthly_train.groupby(['year','month','item_id'],as_index=False)['last_price'].mean()
sales_monthly_train = 


TypeError: incompatible index of inserted column with frame index